In [1]:
! pip install transformers torch datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 42.7 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 39.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 20.6 MB/s  0:00:16m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 29.2 MB/s  0:00:11m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 67.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 56.0 MB/s  0:00:016m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 55.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 25.1 MB/s  0:00:12m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 57.1 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 86.2 MB/s  0:00:00
   ━━━━━

In [2]:
from transformers import pipeline

# Carica il modello
pipe = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", return_all_scores=True)

# Test
text = "I love this product!"
result = pipe(text)
print(result)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
/usr

[[{'label': 'negative', 'score': 0.004656333941966295}, {'label': 'neutral', 'score': 0.010539168491959572}, {'label': 'positive', 'score': 0.9848045110702515}]]


## Fase 2: Valutazione su Dataset Pubblico

In [ ]:
from datasets import load_dataset
from sklearn.metrics import classification_report

dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "all", split="train[:100]")

predictions = []
labels = []

for example in dataset:
    text = example['text']
    label = example['label']
    
    pred = pipe(text)
    pred_label = max(pred[0], key=lambda x: x['score'])['label']
    pred_num = int(pred_label.split('_')[1])
    
    predictions.append(pred_num)
    labels.append(label)

print(classification_report(labels, predictions, target_names=['negative', 'neutral', 'positive']))

## Documentazione

### Scelte Progettuali
- Modello: RoBERTa pre-addestrato per sentiment su Twitter.
- Dataset: Utilizzato dataset multilingue per valutazione.
- Pipeline CI/CD: GitHub Actions per test e deploy.
- Deploy: Gradio su HuggingFace Spaces (alternativa a Streamlit per compatibilità).
- Monitoraggio: MLflow per logging delle metriche.

### Risultati
Il modello mostra accuratezza elevata su testi brevi dei social media.

## Alternativa con Gradio per Deploy su HuggingFace Spaces

In [ ]:
!pip install gradio

import gradio as gr

def analyze_sentiment(text):
    if not text:
        return "Inserisci del testo.", {}
    scores = pipe(text)[0]
    scores_dict = {item['label']: item['score'] for item in scores}
    classification = max(scores_dict, key=scores_dict.get)
    label_map = {"LABEL_0": "Negativo", "LABEL_1": "Neutro", "LABEL_2": "Positivo"}
    classification_it = label_map.get(classification, classification)
    scores_it = {label_map.get(k, k): v for k, v in scores_dict.items()}
    return f"Classificazione: {classification_it}", scores_it

iface = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Inserisci il testo da analizzare..."),
    outputs=[gr.Textbox(label="Risultato"), gr.Label(label="Punteggi")],
    title="Analisi del Sentiment",
    description="Usa Gradio per un'interfaccia web deployabile su HF Spaces."
)

iface.launch(share=True)  # Per demo in Colab